In [1]:
from function import *
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [2]:
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)
sequences, labels = [], []

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [3]:
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            res = np.load(file_path, allow_pickle=True)
            window.append(res)
    sequences.append(window)
    labels.append(label_map[action])

In [4]:
print(sequences)

[[array([ 4.01735634e-01,  7.10041404e-01, -3.13717052e-09,  5.73431551e-01,
        6.70913935e-01, -1.53875023e-01,  7.13818729e-01,  4.62337196e-01,
       -2.03354761e-01,  7.51758695e-01,  2.37602592e-01, -2.37444818e-01,
        7.17756093e-01,  8.06908309e-02, -2.49698520e-01,  6.59469545e-01,
        2.52859116e-01, -6.95957383e-03,  6.43272698e-01,  1.19328246e-01,
       -4.94824424e-02,  6.10421717e-01,  2.75755286e-01, -4.29799184e-02,
        6.08149767e-01,  3.29049706e-01, -4.18171547e-02,  5.18707454e-01,
        2.39275619e-01,  1.32966787e-02,  5.23331821e-01,  1.24404341e-01,
       -7.91932940e-02,  4.86014128e-01,  3.42276275e-01, -1.04327358e-01,
        4.93480951e-01,  3.40480894e-01, -8.66119936e-02,  3.89705777e-01,
        2.35881180e-01,  5.48324361e-03,  3.86812031e-01,  1.30061492e-01,
       -1.06746644e-01,  3.70524287e-01,  3.28165054e-01, -1.25930980e-01,
        3.76135617e-01,  3.62695605e-01, -9.71991271e-02,  2.57811099e-01,
        2.35485062e-01,

In [1]:
X = np.array(sequences)
X

NameError: name 'np' is not defined

In [35]:
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [37]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [38]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(70,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
res = [.7, 0.2, 0.1]

In [41]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])
model.summary()

Epoch 1/300
1/1 [==============================] - 9s 9s/step - loss: 2.5565 - categorical_accuracy: 0.1739
Epoch 2/300
1/1 [==============================] - 0s 119ms/step - loss: 2.4420 - categorical_accuracy: 0.1739
Epoch 3/300
1/1 [==============================] - 0s 126ms/step - loss: 4.5578 - categorical_accuracy: 0.1739
Epoch 4/300
1/1 [==============================] - 0s 154ms/step - loss: 2.3926 - categorical_accuracy: 0.0870
Epoch 5/300
1/1 [==============================] - 0s 172ms/step - loss: 2.7681 - categorical_accuracy: 0.0870
Epoch 6/300
1/1 [==============================] - 0s 151ms/step - loss: 3.0080 - categorical_accuracy: 0.0870
Epoch 7/300
1/1 [==============================] - 0s 233ms/step - loss: 3.1235 - categorical_accuracy: 0.0435
Epoch 8/300
1/1 [==============================] - 0s 159ms/step - loss: 3.1627 - categorical_accuracy: 0.0435
Epoch 9/300
1/1 [==============================] - 0s 142ms/step - loss: 3.1500 - categorical_accuracy: 0.0435
Epoc

In [40]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save('model.h5')